# Import Moduals

In [1]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.losses import MeanSquaredError

from NeuralNetwork import MyModel, MyLayer, InputLayer

# Preprocessing

In [2]:
# import dataset
from keras.datasets import mnist
# load dataset
(x_train, y_train),(x_test, y_test) = mnist.load_data()
# compute the number of labels
num_labels = len(np.unique(y_train))
# convert to one-hot vector
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
# image dimensions (assumed square)
image_size = x_train.shape[1]
input_size = image_size * image_size
input_size
# resize and normalize
x_train = np.reshape(x_train, [-1, input_size])
x_train = x_train.astype('float32') / 255
x_test = np.reshape(x_test, [-1, input_size])
x_test = x_test.astype('float32') / 255
# network parameters
batch_size = 128
hidden_units = 256

In [5]:
x_train.dtype

dtype('float32')

# Keras' Model

In [4]:
model = Sequential()
model.add(Dense(hidden_units, input_dim=input_size, use_bias=False))
model.add(Dense(hidden_units, use_bias=False))
model.add(Dense(num_labels, use_bias=False))
model.compile(loss=MeanSquaredError(), 
              optimizer=SGD(learning_rate=0.01),
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20, batch_size=6000)

Epoch 1/20
10/10 [==============================] - 0s 18ms/step - loss: 0.3288 - accuracy: 0.0804
Epoch 2/20
10/10 [==============================] - 0s 17ms/step - loss: 0.2263 - accuracy: 0.1149
Epoch 3/20
10/10 [==============================] - 0s 18ms/step - loss: 0.1929 - accuracy: 0.1552
Epoch 4/20
10/10 [==============================] - 0s 18ms/step - loss: 0.1710 - accuracy: 0.1984
Epoch 5/20
10/10 [==============================] - 0s 18ms/step - loss: 0.1557 - accuracy: 0.2420
Epoch 6/20
10/10 [==============================] - 0s 18ms/step - loss: 0.1444 - accuracy: 0.2828
Epoch 7/20
10/10 [==============================] - 0s 17ms/step - loss: 0.1357 - accuracy: 0.3197
Epoch 8/20
10/10 [==============================] - 0s 17ms/step - loss: 0.1288 - accuracy: 0.3534
Epoch 9/20
10/10 [==============================] - 0s 18ms/step - loss: 0.1231 - accuracy: 0.3836
Epoch 10/20
10/10 [==============================] - 0s 18ms/step - loss: 0.1183 - accuracy: 0.4110
Epoch 11/

In [5]:
y_hat = model.predict(x_test)
metrics = CategoricalAccuracy()
metrics.update_state(y_test, y_hat)
metrics.result().numpy()

313/313 [==============================] - 0s 705us/step


0.5837

# My Model

In [6]:
initializer = GlorotUniform()

m = MyModel()
m.addLayer(InputLayer(input_size))
m.addLayer(MyLayer(input_size, hidden_units).initializeWeight(initializer))
m.addLayer(MyLayer(hidden_units, hidden_units).initializeWeight(initializer))
m.addLayer(MyLayer(hidden_units, num_labels).initializeWeight(initializer))

#m.selectNodes([list(range(1,10)), list(range(10, 100))])

m.fit(x_train, y_train, 0.01, 20)

c:\Users\wshe801\AppData\Local\anaconda3\envs\CS791-Project\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch  1: loss: 0.449 - accuracy: 0.115
Epoch  2: loss: 0.371 - accuracy: 0.115
Epoch  3: loss: 0.328 - accuracy: 0.114
Epoch  4: loss: 0.302 - accuracy: 0.114
Epoch  5: loss: 0.285 - accuracy: 0.116
Epoch  6: loss: 0.274 - accuracy: 0.118
Epoch  7: loss: 0.265 - accuracy: 0.119
Epoch  8: loss: 0.259 - accuracy: 0.122
Epoch  9: loss: 0.253 - accuracy: 0.124
Epoch 10: loss: 0.247 - accuracy: 0.126
Epoch 11: loss: 0.243 - accuracy: 0.129
Epoch 12: loss: 0.238 - accuracy: 0.132
Epoch 13: loss: 0.234 - accuracy: 0.135
Epoch 14: loss: 0.230 - accuracy: 0.138
Epoch 15: loss: 0.226 - accuracy: 0.140
Epoch 16: loss: 0.223 - accuracy: 0.143
Epoch 17: loss: 0.219 - accuracy: 0.147
Epoch 18: loss: 0.216 - accuracy: 0.150
Epoch 19: loss: 0.213 - accuracy: 0.153
Epoch 20: loss: 0.210 - accuracy: 0.156


In [7]:
y_hat = m.predict(x_test)
metrics.reset_state()
metrics.update_state(y_test, y_hat)
metrics.result().numpy()

0.1604